# Dependencies 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#  Data Frames

In [8]:
companies = pd.DataFrame(pd.read_csv("companies.csv"))
payments = pd.DataFrame(pd.read_csv("payments.csv"))
physicians = pd.DataFrame(pd.read_csv("physicians.csv"))

In [9]:
print(physicians.columns)
print(companies.columns)
print(payments.columns)

Index(['id', 'set', 'First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix',
       'City', 'State', 'Zipcode', 'Country', 'Province', 'Primary_Specialty',
       'License_State_1', 'License_State_2', 'License_State_3',
       'License_State_4', 'License_State_5'],
      dtype='object')
Index(['Company_ID', 'Name', 'State', 'Country'], dtype='object')
Index(['Record_ID', 'Physician_ID', 'Company_ID',
       'Total_Amount_of_Payment_USDollars', 'Date', 'Number_of_Payments',
       'Form_of_Payment_or_Transfer_of_Value',
       'Nature_of_Payment_or_Transfer_of_Value', 'City_of_Travel',
       'State_of_Travel', 'Country_of_Travel', 'Ownership_Indicator',
       'Third_Party_Recipient', 'Charity', 'Third_Party_Covered',
       'Contextual_Information', 'Related_Product_Indicator', 'Product_Code_1',
       'Product_Code_2', 'Product_Code_3', 'Product_Type_1', 'Product_Type_2',
       'Product_Type_3', 'Product_Name_1', 'Product_Name_2', 'Product_Name_3',
       'Product_Category_1', 'Prod

In [10]:
speciality = physicians['Primary_Specialty'].unique().tolist()

In [11]:
temp=payments[payments['Ownership_Indicator']=='Yes']
only_yes=temp['Contextual_Information'].unique().tolist()

In [12]:
import numpy as np
key_nature_1 = 'Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program'
key_nature_2 = 'Current or prospective ownership or investment interest'
key_nature_3 = 'Royalty or License'
ids = physicians['id'].unique().tolist()
payments['Total_Amount_of_Payment_USDollars'] = payments['Total_Amount_of_Payment_USDollars'].fillna(0)
physicians['amount'] = ''
physicians['number_of_payments'] = ''
physicians['payment_form'] = ''
physicians['none_occurance'] = ''
physicians['number_of_companies'] = ''
physicians['has_ownership'] = ''
#physicians['unique_pn'] = ''
#physicians['average_amount'] = ''
physicians['has_key_nature'] = ''
physicians['has_key_context'] = ''


column_list = physicians.columns.tolist()

for i, pid in enumerate(ids):
    pid_frame = payments[payments['Physician_ID']==pid]
    
    list_ownership = pid_frame['Ownership_Indicator'].tolist()
    cindex = column_list.index('has_ownership')
    if 'Yes' in list_ownership or 'yes' in list_ownership :
        physicians.iloc[i,cindex] = 1
    else:
        physicians.iloc[i,cindex] = 0
        
    cindex = column_list.index('amount')   
    physicians.iloc[i,cindex] = pid_frame['Total_Amount_of_Payment_USDollars'].sum()
    cindex = column_list.index('number_of_payments')   
    physicians.iloc[i,cindex] = pid_frame['Number_of_Payments'].sum()
    
    
    form_pay_1=pid_frame[pid_frame['Form_of_Payment_or_Transfer_of_Value']=='Cash or cash equivalent'].shape[0]
    form_pay_2=pid_frame[pid_frame['Form_of_Payment_or_Transfer_of_Value']=='Stock'].shape[0]
    form_pay_3=pid_frame[pid_frame['Form_of_Payment_or_Transfer_of_Value']=='Stock option'].shape[0]
    form_pay_4=pid_frame[pid_frame['Form_of_Payment_or_Transfer_of_Value']=='Dividend, profit or other return on investment'].shape[0]
    form_pay_5=pid_frame[pid_frame['Form_of_Payment_or_Transfer_of_Value']=='Stock, stock option, or any other ownership interest'].shape[0]
    form_pay_6=pid_frame[pid_frame['Form_of_Payment_or_Transfer_of_Value']=='Any other ownership interest'].shape[0]


    cindex = column_list.index('payment_form')
    physicians.iloc[i,cindex] = form_pay_1+form_pay_1+form_pay_2+form_pay_3+form_pay_4+form_pay_5+form_pay_6
    
    cindex = column_list.index('none_occurance')
    physicians.iloc[i,cindex] = pid_frame['Related_Product_Indicator'].tolist().count('None')
    
    cindex = column_list.index('number_of_companies')
    physicians.iloc[i,cindex] = len(pid_frame['Company_ID'].unique().tolist())
    
    pn1 = pid_frame['Product_Name_1'].unique().tolist()
    if 'nan' in pn1:
        unique_pn1 = len(pn1)-1
    else:
        unique_pn1 = len(pn1)-1
    
    pn2 = pid_frame['Product_Name_2'].unique().tolist()
    if 'nan' in pn2:
        unique_pn2 = len(pn2)-1
    else:
        unique_pn2 = len(pn2)-1
    
    pn3 = pid_frame['Product_Name_3'].unique().tolist()
    if 'nan' in pn3:
        unique_pn3 = len(pn3)-1
    else:
        unique_pn3 = len(pn3)-1
    #cindex = column_list.index('unique_pn')
    #physicians.iloc[i,cindex] = unique_pn1+unique_pn2+unique_pn3
    
    #cindex = column_list.index('average_amount')
    #physicians.iloc[i,cindex]=float(pid_frame['Total_Amount_of_Payment_USDollars'].sum()/pid_frame.shape[0])
    
    cindex = column_list.index('has_key_nature')
    nature_list = pid_frame['Nature_of_Payment_or_Transfer_of_Value'].unique().tolist()
    physicians.iloc[i,cindex]=nature_list.count(key_nature_1)+nature_list.count(key_nature_2)+nature_list.count(key_nature_3)
    
    cindex = column_list.index('has_key_context')
    context_list = pid_frame['Contextual_Information'].tolist()
    total_context=0
    for l in only_yes:
        total_context = total_context+context_list.count(l)
    physicians.iloc[i,cindex]=total_context
    
    cindex =  column_list.index('Primary_Specialty')
    physicians.iloc[i,cindex] = speciality.index(physicians.iloc[i,cindex])
    
   


In [14]:
#physicians['average_amount'] = physicians['average_amount'].apply(pd.to_numeric)
physicians['amount'] = physicians['amount'].apply(pd.to_numeric)

In [15]:
physicians[physicians['has_ownership']==0].head(50)

,id,set,First_Name,Middle_Name,Last_Name,Name_Suffix,City,State,Zipcode,Country,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5,amount,number_of_payments,payment_form,none_occurance,number_of_companies,has_ownership,has_key_nature,has_key_context
0,1,train,LEIGH,B,HOPPE,NaN,FORT PIERCE,FL,34950,UNITED STATES,NaN,0,FL,NaN,NaN,NaN,NaN,952.93,47,8,0,18,0,0,46
1,2,train,STEVEN,NaN,FRANK,NaN,TOWSON,MD,21204-6808,UNITED STATES,NaN,1,MD,NaN,NaN,NaN,NaN,120924.92,154,144,0,13,0,1,149
2,3,train,THOMAS,NaN,COUCH,JR.,TROY,NY,12180-2832,UNITED STATES,NaN,2,NY,NaN,NaN,NaN,NaN,3523.93,132,78,0,15,0,0,132
3,4,train,CLAUDE,NaN,SU,NaN,SAVANNAH,GA,31419-1753,UNITED STATES,NaN,3,GA,SC,AL,NaN,NaN,26745.61,411,262,7,32,0,1,386
4,5,train,ROBERT,C,SCHENCK,NaN,ALBUQUERQUE,NM,87131-0001,UNITED STATES,NaN,4,NM,NaN,NaN,NaN,NaN,25411.21,98,32,0,13,0,0,54
6,7,train,CHRISTOPHER,R,GRABER,NaN,SALINA,KS,67401-4189,UNITED STATES,NaN,6,KS,NaN,NaN,NaN,NaN,5822.78,55,6,0,7,0,0,53
7,8,train,DEEPA,S,PATEL,NaN,SPOTSWOOD,NJ,08884-1262,UNITED STATES,NaN,7,NJ,NaN,NaN,NaN,NaN,2396.00,157,0,0,16,0,0,104
8,9,train,CARLA,NaN,OSKA GERAK,NaN,ROYAL OAK,MI,48073-6712,UNITED STATES,NaN,8,MI,NaN,NaN,NaN,NaN,921.98,28,2,0,14,0,0,27
9,10,train,SARANG,N,DESAI,NaN,ALLEN,TX,75013-5289,UNITED STATES,NaN,9,MI,TX,NaN,NaN,NaN,167075.04,141,82,2,34,0,1,121
10,11,train,JAMES,E,RICE,NaN,PINEHURST,NC,28374,UNITED STATES,NaN,10,NC,NaN,NaN,NaN,NaN,12709.72,311,122,2,42,0,0,279


In [16]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

data = physicians[['id', 'set', 'amount', 'number_of_payments','payment_form', 'none_occurance', 
'has_ownership','number_of_companies','has_key_nature','has_key_context','Primary_Specialty']]

scaler = MinMaxScaler()
scaler.fit(data[['amount', 'number_of_payments','payment_form', 'none_occurance', 
'number_of_companies','has_key_nature','has_key_context']])

data[['amount', 'number_of_payments','payment_form', 'none_occurance', 
'number_of_companies','has_key_nature','has_key_context']] = scaler.transform(data[['amount', 'number_of_payments','payment_form', 'none_occurance', 
'number_of_companies','has_key_nature','has_key_context']])
'''
pca = PCA(n_components=9)
pca.fit(data[['amount', 'number_of_payments','payment_form', 'none_occurance', 
'number_of_companies','unique_pn','average_amount','has_key_nature','has_key_context']])
print(pca.explained_variance_ratio_)
data[['amount', 'number_of_payments','payment_form', 'none_occurance', 
'number_of_companies','unique_pn','average_amount','has_key_nature','has_key_context']] = pca.transform(data[['amount', 'number_of_payments','payment_form', 'none_occurance', 
'number_of_companies','unique_pn','average_amount','has_key_nature','has_key_context']])
'''
train_data = data[data['set']=='train'][['id','amount', 'number_of_payments','payment_form', 'none_occurance', 
'has_ownership','number_of_companies','has_key_nature','has_key_context','Primary_Specialty']]
test_data = data[data['set']=='test'][['id','amount', 'number_of_payments','payment_form', 'none_occurance', 
'has_ownership','number_of_companies','has_key_nature','has_key_context','Primary_Specialty']]

train_physicians = train_data['id']
train_data_for_models = train_data[['amount', 'number_of_payments','payment_form', 'none_occurance', 
                                    'number_of_companies','has_key_nature','has_key_context','Primary_Specialty']]
train_labels = train_data['has_ownership']

test_physicians = test_data['id']
test_data_for_models = test_data[['amount', 'number_of_payments','payment_form', 'none_occurance', 
                                   'number_of_companies','has_key_nature','has_key_context','Primary_Specialty']]
test_labels = test_data['has_ownership']

/home/saadi/anaconda3/envs/app/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/saadi/anaconda3/envs/app/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
train_data_for_models.head(10)

,amount,number_of_payments,payment_form,none_occurance,number_of_companies,has_key_nature,has_key_context,Primary_Specialty
0,0.000076,0.011635,0.003010,0.000000,0.209877,0.000000,0.016502,0
1,0.010946,0.043556,0.054176,0.000000,0.148148,0.333333,0.054272,1
2,0.000309,0.036993,0.029345,0.000000,0.172840,0.000000,0.048038,2
3,0.002413,0.120227,0.098570,0.032710,0.382716,0.333333,0.141181,3
4,0.002292,0.026850,0.012039,0.000000,0.148148,0.000000,0.019435,4
5,0.000233,0.007757,0.004515,0.023364,0.148148,0.333333,0.010634,5
6,0.000517,0.014021,0.002257,0.000000,0.074074,0.000000,0.019069,6
7,0.000207,0.044451,0.000000,0.000000,0.185185,0.000000,0.037770,7
8,0.000073,0.005967,0.000752,0.000000,0.160494,0.000000,0.009534,8
9,0.015127,0.039678,0.030850,0.009346,0.407407,0.333333,0.044004,9


In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

oversample = SMOTE(k_neighbors=10, random_state=42)
X, y = oversample.fit_resample(train_data_for_models, train_labels)

rf_model =  RandomForestClassifier(max_depth=None, n_estimators=300, random_state=42)
rf_model.fit(X,y)


RandomForestClassifier(n_estimators=300, random_state=42)

In [46]:
rf_model.feature_importances_

array([0.09939601, 0.08556042, 0.15258127, 0.16660958, 0.09372548,
       0.14552599, 0.08076417, 0.17583707])

In [47]:
pred = rf_model.predict(train_data_for_models).tolist()
print(classification_report(train_labels, pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4700
           1       1.00      1.00      1.00       300

    accuracy                           1.00      5000
   macro avg       1.00      1.00      1.00      5000
weighted avg       1.00      1.00      1.00      5000



In [48]:
pred = rf_model.predict(test_data_for_models).tolist()
print(pred.count(1))
test_physicians = test_data['id'].tolist()

79


In [49]:
import csv
csv_file_columns = ['id','prediction']
csv_file = open("submission_intellectual_koala.csv", 'w')
csv_writer = csv.DictWriter(csv_file, fieldnames = csv_file_columns)
csv_writer.writeheader()

for pid, prediction in zip(test_physicians, pred):
    row = {'id' : pid, 'prediction' : prediction }
    csv_writer.writerow(row)
csv_file.close()